Dans ce TP, nous considérons des trajets en vélo partagé (similaire au Vélib) en Californie. Deux jeux de données sont fournis : l'un qui contient les stations de vélo, l'autre, les trajets à vélo. Les déplacements à vélo se font d'une station à l'autre.

In [2]:
import pandas as pd
import sqlite3

Charger le fichier stationData.csv dans un DataFrame `station_df` et le fichier tripData.csv dans un DataFrame `trip_df`. Pour chaque DataFrame, il vous faudra demander une inférence des schémas et indiquer que la première ligne est un en-tête.

In [3]:
trip_df = pd.read_csv("../../data/tripData.csv")
station_df = pd.read_csv("../../data/stationData.csv")

Afficher les schémas/informations des deux DataFrames.

In [4]:
station_df.head(10)

,station_id,name,lat,long,dockcount,landmark,installation
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


In [5]:
trip_df.head()

,TripID,Duration,StartDate,StartStation,StartTerminal,EndDate,EndStation,EndTerminal,Bike#,SubscriberType,ZipCode
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
3,913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
4,913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


In [6]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354152 entries, 0 to 354151
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   TripID          354152 non-null  int64 
 1   Duration        354152 non-null  int64 
 2   StartDate       354152 non-null  object
 3   StartStation    354152 non-null  object
 4   StartTerminal   354152 non-null  int64 
 5   EndDate         354152 non-null  object
 6   EndStation      354152 non-null  object
 7   EndTerminal     354152 non-null  int64 
 8   Bike#           354152 non-null  int64 
 9   SubscriberType  354152 non-null  object
 10  ZipCode         353874 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [8]:
station_df = "toto"

In [10]:
print(station_df)

toto


In [21]:
station_df.describe()

,station_id,lat,long,dockcount
count,70.000000,70.000000,70.000000,70.000000
mean,43.000000,37.590243,-122.218416,17.657143
std,24.166092,0.203473,0.209446,4.010442
min,2.000000,37.329732,-122.418954,11.000000
25%,24.250000,37.389483,-122.400601,15.000000
50%,43.500000,37.631163,-122.312123,15.000000
75%,63.750000,37.788123,-122.078009,19.000000
max,84.000000,37.804770,-121.877349,27.000000


In [25]:
trip_df.describe().round(2).style.format(lambda x: f"{x:,}".replace(",", " ") if isinstance(x, (int, float)) else x)

,TripID,Duration,StartTerminal,EndTerminal,Bike#
count,354 152.0,354 152.0,354 152.0,354 152.0,354 152.0
mean,676 962.27,1 046.03,58.45,58.42,422.96
std,138 874.15,30 016.94,16.74,16.88,159.84
min,432 947.0,60.0,2.0,2.0,9.0
25%,557 009.75,342.0,50.0,50.0,327.0
50%,679 451.5,511.0,62.0,63.0,437.0
75%,798 026.25,739.0,70.0,70.0,546.0
max,913 460.0,17 270 400.0,84.0,84.0,878.0


Créer une base SQLite ainsi qu'une vue SQL pour chaque DataFrame.

In [33]:
conn = sqlite3.connect(":memory:")
station_df.to_sql("station", conn, if_exists="replace")

70

In [34]:
trip_df.to_sql("trip", conn, if_exists="replace")

354152

Trouver deux façons de calculer le nombre de trajets, l'une en appelant une méthode sur `trip_df` directement, l'autre en rédigeant une requête SQL sur la vue correspondante.

In [36]:
# Méthode SQL
query = "select count(*) from trip"
pd.read_sql(query, conn)

,count(*)
0,354152


In [37]:
# Méthode pandas
len(trip_df)

354152

In [39]:
trip_df["TripID"].count()

354152

Écrire une requête permettant de compter le nombre de trajets qui démarrent et se terminent à la même station.

In [ ]:
mask = trip_df["StartTerminal"] == trip_df["EndTerminal"]

In [ ]:
trip_df["circular"] = mask

In [54]:
trip_df["circular"].value_counts()

circular
False    343876
True      10276
Name: count, dtype: int64

Obtenir l’id des stations associées à ces trajets. Écrire une requête renvoyant la liste des terminaux concernés ainsi que le nombre de trajets pour chacun de ces terminaux. Trier le résultat par ordre décroissant de nombre de trajets.

In [61]:
circular_per_station_df = trip_df[
    mask
].groupby(
    "StartTerminal"
).size(
).reset_index(
    name="trip_count"
).sort_values(
    "trip_count", ascending=False
)

In [62]:
display(circular_per_station_df)

,StartTerminal,trip_count
48,60,850
40,50,708
28,35,348
64,76,320
62,74,307
...,...,...
66,80,26
17,24,23
68,83,21
14,21,12


Compléter le résultat en indiquant le nombre de docks (dockcount) des stations concernées.

In [ ]:
circular_per_station_df.merge(
    station_df[
        ["station_id", "dockcount"]
    ],
    left_on="StartTerminal",
    right_on="station_id"
).drop("StartTerminal", axis=1)

,trip_count,station_id,dockcount
0,850,60,15
1,708,50,23
2,348,35,11
3,320,76,19
4,307,74,23
...,...,...,...
65,26,80,15
66,23,24,15
67,21,83,15
68,12,21,15


In [2]:
animals = ['giraffe', 'tiger', 'monkey', 'mouse']
animals[3:2]

[]